In [1]:
import torch
from torch.utils.data.dataset import Dataset, TensorDataset
from torch.utils.data.dataloader import DataLoader
from DiacriticDataset import DiacriticDatasetShaddah
import pickle
from scipy.sparse import csr_matrix
import numpy as np
from scipy.optimize import fmin_l_bfgs_b

In [2]:
train_dataset = DiacriticDatasetShaddah('Dataset_with_shaddah/train_cleaned_withshadda_215.txt','Dataset_with_shaddah/letter_to_id.pickle','Dataset_with_shaddah/id_to_letter.pickle','Dataset_with_shaddah/diacritic_to_id.pickle','Dataset_with_shaddah/id_to_diacritic.pickle','Dataset_with_shaddah/word_to_id.pickle','Dataset_with_shaddah/id_to_word.pickle','Dataset/diacritic_to_id.pickle','Dataset/id_to_diacritic.pickle','Dataset_with_shaddah/word_to_id_withdiacritics.pickle','Dataset_with_shaddah/id_to_word_withdiacritics.pickle')

In [3]:
train_dataloader = DataLoader(train_dataset)
Y = train_dataset.id_to_diacritic.keys()
with open('Dataset_with_shaddah/diacritic_to_id_memm.pickle','rb') as file:
    diacritic_to_id_memm = pickle.load(file)

In [ ]:
def L_Lprime(v,*args):
    lamda = args[0]
    training_matrix = None #must fill
    num_of_features = None #must fil
    first_summerand_L= training_matrix.dot(v).sum()
    second_summerand_L  = 0
    
    first_summerand_Lprime = np.array(training_matrix.sum(axis=0))[0]
    second_summerand_Lprime = csr_matrix((1, num_of_features), dtype=np.int8)

    for training_example in train_dataloader:
        letters = training_example[0][0].tolist()
        diacritics = training_example[1][0].tolist()
        
        for position,letter in enumerate(letters):
            inner_summation = 0
            
            if position==0:
                history = History(diacritic_to_id_memm['*'],diacritic_to_id_memm['*'],letters,position)
            if position==1:
                history = History(diacritics[position-1],diacritic_to_id_memm['*'],letters,position)
            else:
                history = History(diacritics[position-1],diacritics[position-2],letters,position)
            
            feature_summation = csr_matrix((1, num_of_features), dtype=np.int8)
            for tag in Y:
                features = generate_features(history,tag)
                
                dot_product = np.exp(features.dot(v))[0]
                
                feature_summation += features*dot_product
                
                
                inner_summation += dot_product
                
                
            second_summerand_L += np.log(inner_summation)   
            
            feature_summation /= inner_summation
            
            second_summerand_Lprime += feature_summation
    
    regularization = (lamda/2)*np.inner(v,v)
    regularization_prime = lamda*v
    L = -(first_summerand - second_summerand-regularization)     
    Lprime = -(first_summerand_Lprime-second_summerand_Lprime.toarray()[0]-regularization_prime)
    return L,Lprime

In [ ]:
num_of_features = None #must fill
lamda = 1
maxiterations = 1
_iprint = 1
v0 = np.random.rand(num_of_features)
v_optimal,f,d = fmin_l_bfgs_b(func=L_Lprime,x0=v0,fprime=None,args=(lamda),maxiter=maxiterations,iprint=_iprint)
weights = v_optimal[0]

In [14]:
training_matrix = csr_matrix([[1, 2, 0], [0, 0, 3], [4, 0, 5]])
v = np.array([1, 0, -1])
num_of_features = 3
second_summerand_Lprime = csr_matrix((1, num_of_features), dtype=np.int8)
feature_summation= csr_matrix((1, num_of_features), dtype=np.int8)
test = [csr_matrix([[1, 2.5, 0]]),csr_matrix([[1, 2, 0]])]
first_summerand_L= training_matrix.dot(v).sum()
second_summerand_L  = 0

first_summerand_Lprime = np.array(training_matrix.sum(axis=0))[0]
second_summerand_Lprime = csr_matrix((1, num_of_features), dtype=np.int8)
inner_summation = 0
for features in test:
    dot_product = np.exp(features.dot(v))[0]
    feature_summation += features*dot_product
    inner_summation += dot_product
    
second_summerand_L += np.log(inner_summation)
feature_summation /= inner_summation
second_summerand_Lprime += feature_summation

In [15]:
second_summerand_Lprime.toarray()

array([[1.  , 2.25, 0.  ]])